In [ ]:
import math
from datetime import date

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, pivot_table, read_csv
from scipy.signal import argrelextrema

from rxtrade.data import add_amplitude_column, add_basic_columns
from rxtrade.utils import get_data_files

In [94]:
def get_data_df(ticker: str, file_path: str) -> DataFrame:
    df = read_csv(file_path)

    add_basic_columns(df)
    add_amplitude_column(ticker, df)

    # df["Wave Min"] = df.iloc[argrelextrema(df["close"].values, np.less_equal, order=3)[0]]["close"]
    # df["Wave Max"] = df.iloc[argrelextrema(df["close"].values, np.greater_equal, order=3)[0]]["close"]

    df[f"Body / Wick ({ticker})"] = abs(df["close"] - df["open"]) / abs(df["high"] - df["low"])
    df["Prev Open"] = df["open"].shift(1)
    df["Prev High"] = df["high"].shift(1)
    df["Prev Low"] = df["low"].shift(1)
    df["Prev Close"] = df["close"].shift(1)
    df["Prev Big Action"] = df[f"Amplitude ({ticker})"].shift(1) > (df["Amplitude (NQ - 5)"].shift(1) * 1.3)

    # Passive (high body/wick rate)
    # CLOSE did not break prev OPEN - HOLD
    # CLOSE breaks prev OPEN - change side
    # > Danger in CSLD
    # -----------------
    # Aggressive (low body/wick rate)
    # CURRENT did not break prev OPEN - HOLD
    # CURRENT breaks prev OPEN - change side
    # CLOSE did not break prev OPEN - get position back
    # -----------------
    # Take Profit
    # - # Ks not breaking HIGH or LOW
    # - Trendline

    return df

In [95]:
df = get_data_df("NQ", "data/CME_MINI_NQ1!, 5.csv")
df.set_index("time", inplace=True)

df

,open,high,low,close,VWAP,MA #1,MA #2,MA #3,BB Basis,BB #1 Upper,...,Market Hour,Market Code,Amplitude (NQ),Amplitude (NQ - 5),Body / Wick (NQ),Prev Open,Prev High,Prev Low,Prev Close,Prev Big Action
time,,,,,,,,,,,,,,,,,,,,,
2022-02-03 14:00:00-06:00,14591.00,14607.00,14569.00,14580.00,14728.148875,14589.476618,14605.976727,14675.363852,14640.8625,14718.415139,...,14,R,38.00,33.392749,0.289474,NaN,NaN,NaN,NaN,False
2022-02-03 14:05:00-06:00,14580.25,14601.50,14575.50,14582.75,14727.100983,14587.234412,14601.753686,14672.327332,14635.7875,14714.593000,...,14,R,26.00,31.089123,0.096154,14591.00,14607.00,14569.00,14580.00,False
2022-02-03 14:10:00-06:00,14583.25,14607.25,14573.75,14600.50,14725.993668,14591.656275,14601.525743,14669.972337,14631.0875,14706.463603,...,14,R,33.50,33.633685,0.514925,14580.25,14601.50,14575.50,14582.75,False
2022-02-03 14:15:00-06:00,14599.75,14623.00,14599.25,14619.75,14725.003331,14601.020850,14604.839244,14668.325703,14628.8625,14702.805126,...,14,R,23.75,33.700527,0.842105,14583.25,14607.25,14573.75,14600.50,False
2022-02-03 14:20:00-06:00,14619.00,14640.75,14611.25,14612.50,14724.019777,14604.847233,14606.232109,14666.495352,14626.2375,14698.572221,...,14,R,29.50,38.675790,0.220339,14599.75,14623.00,14599.25,14619.75,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-04 15:35:00-06:00,14644.75,14651.00,14642.75,14645.00,14619.066287,14648.059427,14664.560742,14681.684043,14717.1250,14847.824512,...,15,E,8.25,8.182099,0.030303,14643.25,14647.00,14638.00,14645.25,False
2022-02-04 15:40:00-06:00,14644.75,14648.50,14641.00,14647.50,14619.074875,14647.872951,14661.458789,14680.563255,14710.0875,14839.864354,...,15,E,7.50,8.148148,0.366667,14644.75,14651.00,14642.75,14645.00,False
2022-02-04 15:45:00-06:00,14647.75,14651.00,14641.75,14644.75,14619.084184,14646.831968,14658.420827,14679.389050,14702.8375,14830.180960,...,15,E,9.25,8.472222,0.324324,14644.75,14648.50,14641.00,14647.50,False


In [106]:
history = []

def long(time, price: float):
    if history and history[-1]["side"] == "LONG":
        return

    history.append({"time": time, "side": "LONG", "price": price})

def short(time, price: float):
    if history and history[-1]["side"] == "SHORT":
        return

    history.append({"time": time, "side": "LONG", "price": price})

def reverse_position(time, price: float):
    if history and history[-1]["side"] == "LONG":
        return

    long(time, price)

def is_prev_up(row):
    return row["Prev Close"] > row["Prev Open"]

def is_prev_down(row):
    return row["Prev Close"] < row["Prev Open"]

def is_prev_reversed(row):
    if row["close"] > row["open"]:
        return is_prev_down(row)

    return is_prev_up(row)

def is_current_over_prev(row, mult: float = 1):
    if not is_prev_reversed(row):
        return

    match (is_prev_up(row), mult):
        case (True, 1):
            return row["close"] < row["Prev Open"]
        case (False, 1):
            return row["close"] < row["Prev Open"]
        case (True, mult):
            return row["close"] < (row["open"] - abs(row["Prev Close"] - row["Prev Open"]) * mult)
        case (False, mult):
            return row["close"] > (row["open"] + abs(row["Prev Open"] - row["Prev Close"]) * mult)

for time, row in df.iterrows():
    if math.isnan(row["Prev Open"]):
        # History #1 - observe
        continue

    if not history:
        # History #2 - initial entry
        if is_prev_up(row):
            long(time, row["open"])
        else:
            short(time, row["open"])

        continue

    # History #3+
    mult = 0.6 if row["Prev Big Action"] else 1

    if not is_current_over_prev(row, mult):
        continue

    if is_prev_up(row):
        reverse_position(time, row["open"] - abs(row["Prev Close"] - row["Prev Open"]) * mult)
    else:
        reverse_position(time, row["open"] + abs(row["Prev Open"] - row["Prev Close"]) * mult)

DataFrame(history)

NameError: name 'math' is not defined

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_310_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_win32_310_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 2019.3.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 2019.3.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 2019.3.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [83]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
